# Basic Multi-Agent Example with AutoGen

This notebook implements a simple multi-agent system using AutoGen. 

In [1]:
import os, nest_asyncio
nest_asyncio.apply()
import asyncio
import timeit
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.tools import TeamTool
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.curdir, "..")))

from tools import (
    create_chat_completion_client,
    current_datetime_utc_tool,
    adjust_properties_for_fixed_response,
    get_tavily_search_tool,
    get_attributes,
    load_model_config,
)

#### Note: You need a tavily API key to use the web search tool!

In [2]:
model = load_model_config() 

# Check if web search is possible (i.e., if API key is set)
ALLOW_WEB_SEARCH = os.getenv("TAVILY_API_KEY", False)

Using model: gpt-4.1-nano type: openai


In [3]:
# Prepare tools
basic_tools = {current_datetime_utc_tool}
web_search_tools = set()
if ALLOW_WEB_SEARCH:
    # web_search_tool = [ await get_rag_web_browser_tool() ]
    web_search_tools = { await get_tavily_search_tool() }

Set the joke's topic
----------------------

In [4]:
topic = "Animals"

Select Model and Agent Configuration
-----------------------------

In [ ]:
agentConfig = {
    "Author": dict(
        system_message="Write a SHORT joke above the provided topic."
    ),
    "SmartassEditor": dict(
        system_message="Analyze the joke, explain why it is funny or not funny, and make it funnier without making it longer.",
        tools=web_search_tools,
    ),
}

# For testing, we can fix an agent's response to a known answer
#adjust_properties_for_fixed_response(agentConfig["Author"], "What do you call a magic dog? A Labracadabrador.")

In [6]:
# Now create model client
model_client = create_chat_completion_client(model)

agents = []  # start with a user proxy for the researcher

for agent_type, props in agentConfig.items():
    agents.append(
        AssistantAgent(
            name=agent_type,
            **props,
            model_client=model_client,
        )
    )

In [7]:
# Set up termination conditions
termination = MaxMessageTermination(max_messages=10) #len(agents))

# Create the team
team = RoundRobinGroupChat(
    participants=agents,
    termination_condition=termination,
    max_turns=len(agents) # we don't really want it to go round
)

In [8]:
# Create the host agent with deep dive capabilities and the host agent
joker_tool = TeamTool(team=team, name="JokeGenerationTeam", description="Collaborative multi-agent joke generation team.")
host = AssistantAgent(
    name="Host",
    model_client=model_client,
    tools=[joker_tool],
    system_message="You have access to a Joke Generation team for short animal-related jokes."
)

Start the System
===================

In [9]:
async def invent_joke(topic: str):
    start_time_s = timeit.default_timer()
    result = await host.run(task=f"Invent a short joke about: {topic}")
    elapsed = timeit.default_timer()-start_time_s
    print(f"🔍 Generation result (took {elapsed:.2f} sec.) resulted in {len(result.messages)} messages.")
    return result  # Return the result for further exploration

# Run the deep dive
result = await invent_joke(topic)

🔍 Generation result (took 2.20 sec.) resulted in 9 messages.


Explore Results
===============

In [10]:
# Display the full result structure
from json import tool


print("Result type:", type(result))

print("\nResult attributes:")
print(get_attributes(result))
print("\nMessage attributes:")
print(get_attributes(result.messages[-1]))

for i, msg in enumerate(result.messages):
    tool_info = f"Tools: {[c.name for c in msg.content if hasattr(c, 'name')]}" if "tool" in msg.type.lower() else "no tool involved"
    if hasattr(msg, 'tool_calls'):
        tool_info += f", tool_calls: {getattr(msg, 'tool_calls', None)}"
    ts = msg.created_at.strftime("%H:%M:%S.%f")[:-3] if getattr(msg, "created_at", None) else "(unknown)"
    #print("")
    print(f"{i}: {msg.type} by {msg.source} at {ts}, {tool_info}")
    #print(msg.content)

Result type: <class 'autogen_agentchat.base._task.TaskResult'>

Result attributes:
['construct', 'copy', 'dict', 'from_orm', 'json', 'messages', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_strings', 'parse_file', 'parse_obj', 'parse_raw', 'schema', 'schema_json', 'stop_reason', 'update_forward_refs', 'validate']

Message attributes:
['construct', 'content', 'copy', 'created_at', 'dict', 'dump', 'from_orm', 'id', 'json', 'load', 'metadata', 'model_computed_fields', 'model_config', 'model_construct', 'model_copy', 'model_dump', 'model_dump_json', 'model_extra', 'model_fields', 'model_fields_set', 'model_json_schema', 'model_parametrized_name', 'model_post_init', 'model_rebuild', 'model_validate', 'model_validate_json', 'model_validate_st

In [11]:
# Clean up resources
await model_client.close()

In [12]:
import re
from autogen_agentchat.messages import BaseTextChatMessage

print("Stop reason:", result.stop_reason)
assert isinstance(result.messages[-1], BaseTextChatMessage)
last_message = result.messages[-1].content

print("Message Sequence:")
agent_names = list(agentConfig.keys())
pattern = re.compile(r'(' + '|'.join(re.escape(n) for n in agent_names) + r'):')
for match in pattern.finditer(last_message):
    print(" -", match.group(1))

print("\n", last_message)  # Print the content of the last message

Stop reason: None
Message Sequence:
 - Author
 - SmartassEditor

 Author: What do you call a magic dog? A Labracadabrador.

SmartassEditor: This joke is funny because it combines a well-known dog breed, Labrador, with the word "abracadabra," a common magic word, creating a silly and memorable pun. It plays on the expectation of a typical animal joke while adding a clever twist with wordplay.

To make it funnier without lengthening it, I could tweak it slightly for a more surprising punchline:

**What do you call a dog that performs magic? A Labracadabra!**
